In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import neurokit2 as nk
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import AdaBoostClassifier
import pyhrv.time_domain as td
import pywt
import ipynb
import matplotlib.pyplot as plt
import pickle
import pyhrv
import ipynb
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# functions for features extraction
from ipynb.fs.full.ECG_features import get_ecgfeatures
from ipynb.fs.full.EDA import get_edaindex, get_edafeatures
from ipynb.fs.full.BVP import get_bvpfeatures, bvp_prep

c:\Users\riche\AppData\Local\Programs\Python\Python39\lib\site-packages\antropy\fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit('float64(float64[:], int32)')


Get train data from dataset

In [50]:
features = pd.read_csv("out_features.csv", index_col=0)
features = bvp_prep(features)

c:\Users\riche\OneDrive\Documenten\GitHub\EPO4-C2\BVP.ipynb:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    features_event.reset_index(drop=True, inplace=True)\n",


In [51]:
num_features = 25
y = features['y']
X_total = features.drop(['y', 'index', 'PPG_Rate_Mean'], axis=1)
print(len(y), len(X_total))

402 402


Feature selection

In [52]:
# LASSO
num_feats = num_features
scaler = StandardScaler().fit(X_total)
X_train = scaler.transform(X_total)
embeded_lr_selector = SelectFromModel(LogisticRegression(C=1, penalty="l1", solver='liblinear'), max_features=num_feats)
embeded_lr_selector.fit(scaler.transform(X_total), y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X_total.loc[:,embeded_lr_support].columns.tolist()

Classifier

In [53]:
def svm_func(X_train, y_train):
    # define the model
    svm_model = svm.SVC()

    # fit/train the model on all features
    svm_model.fit(X_train, y_train)
    # Y_test_pred = svm_model.predict(X_test)

    # # score
    # svm_score = svm_model.score(X_test,y_test)
    return svm_model #, Y_test_pred, svm_score

Create model

In [64]:
if "PPG_Rate_Max" in embeded_lr_feature:
        embeded_lr_feature.remove("PPG_Rate_Max")

X_train = features
y_train = X_train['y']
X_train = X_train.drop(['y', 'index'], axis=1)
X_train = X_train[list(embeded_lr_feature)]

# fit the data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# Get model
svm_model = svm_func(X_train, y_train)
# Get the score

Read measured data from csv

In [56]:
columns_ecg =['index', 'ecg_HR_mean', 'ecg_HR_min', 'ecg_HR_max', 'ecg_HR_std', 'ecg_SDNN', 'ecg_SDANN', 'ecg_RMSSD', 
                                           'ecg_SDSD','ecg_pNN50', 'ecg_pNN20', "ecg_triangular_index", "ecg_tinn", "ecg_sd1", "ecg_sd2",
                                            "ecg_ratio_sd2_sd1", 'ecg_abs_power_VLF', 'ecg_abs_power_LF', 'ecg_abs_power_HF', 'ecg_tot_power',
                                            'ecg_LF/HF', 'ecg_peak_vlf', 'ecg_peak_lf', 'ecg_peak_hf', 'ecg_norm_power_LF', 'ecg_norm_power_HF',
                                            'max_cwt_absmean', 'min_cwt_absmean', 'mean_cwt_absmean','std_cwt_absmean', 
                                            'max_cwt_std', 'min_cwt_std', 'mean_cwt_std','std_cwt_std',
                                            'max_cwt_energy', 'min_cwt_energy', 'mean_cwt_energy','std_cwt_energy',
                                            'max_cwt_var', 'min_cwt_var', 'mean_cwt_var','std_cwt_var',
                                            'entropy']
columns_resp = ['RSP_Rate_Mean','RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB', 'RRV_SDSD', 'RRV_CVBB', 'RRV_CVSD', 'RRV_MedianBB',
                'RRV_MadBB', 'RRV_MCVBB', 'RRV_VLF', 'RRV_LF', 'RRV_HF', 'RRV_LFHF', 'RRV_LFn', 'RRV_HFn', 'RRV_SD1',
                'RRV_SD2', 'RRV_SD2SD1', 'RRV_ApEn', 'RRV_SampEn', 'RSP_Amplitude_Mean', 'RSP_RVT', 'RSP_Symmetry_PeakTrough',
                'RSP_Symmetry_RiseDecay', 'RSP_Phase_Duration_Inspiration', 'RSP_Phase_Duration_Expiration', 'RSP_Phase_Duration_Ratio']

columns = [*columns_ecg, *columns_resp]
meas_ecgfeatures = pd.DataFrame(None, columns= columns)
meas_edafeatures = None
meas_bvpfeatures = None

fs = 400 # sampling frequency
duration = 45

In [57]:
data = pd.read_csv("Daan-nultest_2.csv")

ecg_base = data['ECG Data']
eda_base = data['GSR Data']
t = data['TimeStamp']
# bvp = PPG_Red Data,PPG_IR Data

ecg_base = ecg_base[9:]
eda_base = eda_base[9:]

# Multiple smaller samples per signal
# samples = 10 # specify amount of samples
n_samples = duration * fs
for j in range(t[len(t)-1]//n_samples): # loop over all segments
    # Cutting the signal into a segment
    start, end = j*fs*duration, (j+1)*fs*duration
    ecg = ecg_base[(ecg_base > start) & (ecg_base < end)]
    eda = eda_base[(eda_base > start) & (eda_base < end)]
    # bvp = bvp_base[start: end]

    # getting the eda and ecg features
    index = str(j)
    tonic, phasic, start, end = get_edaindex(eda, fs)
    meas_edafeatures = get_edafeatures(index, meas_edafeatures, phasic, tonic, fs)
    meas_ecgfeatures = get_ecgfeatures(ecg, fs, meas_ecgfeatures, index)
    
    # bvpfeature = get_bvpfeatures(bvp, fs_bvp, index)
    # bvpfeatures = pd.concat([bvpfeatures, bvpfeature], axis=0)

In [58]:
# bvp_filt = bvp_prep(bvpfeatures)

features1 = pd.merge(meas_ecgfeatures, meas_edafeatures, left_index=True, right_index=True)
# total_features = pd.merge(bvp_filt, features1, on='index')
features1
# total_features = total_features[total_features.RSP_RVT != 0.0]

,index,ecg_HR_mean,ecg_HR_min,ecg_HR_max,ecg_HR_std,ecg_SDNN,ecg_SDANN,ecg_RMSSD,ecg_SDSD,ecg_pNN50,...,scr_Entropy,scr_max_freq,scl_mean,scl_std,scl_max,scl_min,scl_range,scl_kurtosis,scl_skew,scl_momentum
0,0,75.784329,65.934066,90.566038,6.226686,0.078104,0.0,0.067202,53.534814,0.25,...,952236.153605,0.0,81.066552,14.606922,114.967428,64.340333,50.627095,-0.263396,0.994003,213.362159


In [62]:
for feat in embeded_lr_feature:
    if feat not in features1:
        embeded_lr_feature.remove(feat)

meas_features = features1[embeded_lr_feature]

HRV_MFDFA_alpha2_Delta


In [65]:
svm_model.predict(meas_features)

c:\Users\riche\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


array([1], dtype=int64)